<a href="https://colab.research.google.com/github/Hemachandran-D45/vectordb/blob/main/RAG_Pipeline_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community -q
!pip install langchain_huggingface -q
!pip install huggingface_hub -q
!pip install langchain_core -q

In [ ]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("/content/RohitSharma.txt")
loader.load()

In [5]:
rohit_doc = loader.load()

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

In [8]:
documents =text_splitter.split_documents(rohit_doc)

In [9]:
#for i, chunk in enumerate(documents):
 # print(f"Chunk {i+1} (Length: {len(chunk.page_content)} character)")
  #print(chunk.page_content)

In [16]:
!pip install chromadb -q

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
db = Chroma.from_documents(documents, HuggingFaceEmbeddings())

In [20]:
import os
from langchain.llms import HuggingFaceHub
from google.colab import userdata
sec_key = userdata.get("RAG_Embedding")

In [24]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = sec_key
repo_id = "google/flan-t5-xl"
task = "text2text-generation"


In [25]:
llm = HuggingFaceHub(repo_id=repo_id, task=task)

In [33]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the question based only on provided information.
<context>{context}</context>
Question: {input}""")

In [34]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)


In [46]:
retriever = db

In [47]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [48]:
retrieval_chain.invoke({"input":"You won the T20 World Cup 2024"})

BadRequestError:  (Request ID: 1a-qC_2meuLmEYhYi3tVt)

Bad request:
Authorization header is correct, but the token seems invalid